# Gym demo

Demonstrate the use of the the Gym interface and screen capture support

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import sys
import os
import numpy as np
import pandas as pd
import glob as glob
import time
import pickle
from omegaconf import OmegaConf
import cv2

# add custom paths
sys.path.extend([os.path.abspath('./assetto_corsa_gym')])
import AssettoCorsaEnv.assettoCorsa as assettoCorsa

# Configure the logging system
import logging
logger = logging.getLogger(__name__)
logging.basicConfig(
    level=logging.INFO,  # Set the logging level (DEBUG, INFO, WARNING, ERROR, CRITICAL)
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',  # Format of the log messages
    datefmt='%Y-%m-%d %H:%M:%S',  # Format of the timestamp
)



## Load config file

In [ ]:
config = OmegaConf.load("config.yml")

config.AssettoCorsa.screen_capture_enable = True

# Create env object

In [ ]:
env = assettoCorsa.make_ac_env(cfg=config, work_dir="output")


# Show static info

In [ ]:
static_info = env.client.simulation_management.get_static_info()
ac_mod_config = env.client.simulation_management.get_config()

logger.info("Static info:")
for i in static_info:
    logger.info(f"{i}: {static_info[i]}")
logger.info("AC Mod config:")
for i in ac_mod_config:
    logger.info(f"{i}: {ac_mod_config[i]}")


# Apply actions and recover the car

In [ ]:
states = []

env.reset()

for i in range(100):
    if i % 2 == 0:
        steer = .1
    else:
        steer = -.1
    env.set_actions(np.array([steer, 0.5, -1.]))
    next_state, reward, done, info = env.step(action=None)  # action is already applied
    if (i % 25) == 0:
        plt.imshow(cv2.cvtColor(env.get_current_image(), cv2.COLOR_BGR2RGB))
        plt.show()
    time.sleep(0.01)
    if done:
        break

env.recover_car()
env.close()

# Plot states

In [ ]:
df = env.get_history()
df.columns

In [ ]:
plt.title("Speed")
plt.plot(df.speed)
plt.show()

plt.title("steerAngle")
plt.plot(df.steerAngle)
plt.show()